In [1]:
'''
Synthisizing Fingerprint with WGAN based on Crossmatch
'''
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Conv2D,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D,LeakyReLU
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt
testname= 'WGAN'
if not os.path.exists('/home/cc/Data/'+testname):
    os.makedirs('/home/cc/Data/'+testname)

In [2]:
def rescaleimage(image):
    image=(image-127.5)/127.5
    return image

In [3]:
data_pd = pd.read_pickle('/home/cc/Data/Zips/cm_index_df_fixed.p')
data_pd['FilePath']= data_pd.FilePath.apply(lambda x: os.path.join('/home/cc/Data/Images',x))
data_live = data_pd[data_pd.live_spoof == 0] # only live samples
data_live['Collection'] = data_live.CaptureID.apply(lambda x: x[0:3])
resize_df = data_live[data_live.Collection =='Pre']
resize_df.shape

/home/cc/anaconda3/envs/Clarkson/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


(236345, 21)

In [8]:
class WGAN():
    def __init__(self):
        self.img_rows = 512
        self.img_cols = 512
        self.channels = 3 # I am using channel last models
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 512
        self.batch_size=256
        self.Ngpus = 1
        self.inputTrain = '/home/cc/Data/Images'
        self.Imsize=(self.img_cols,self.img_cols)
        
        datagen=ImageDataGenerator(preprocessing_function=rescaleimage) # we rescale to -1 to 1 for our tanhs 
        self.image_generator= datagen.flow_from_dataframe(
                                directory= self.inputTrain,
                                dataframe = resize_df,
                                x_col = 'FilePath',
                                color_mode='rgb',
                                target_size=(self.img_rows,self.img_cols),
                                batch_size=self.batch_size,
                                class_mode=None,
                                shuffle=True,
                                )
        
        self.len=len(self.image_generator)
        self.noise = np.random.normal(0, 1, (5 * 5, self.latent_dim))

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 3
        self.clip_value = 0.01
        #optimizer = RMSprop(lr=0.00005)
        optimizer=Adam(lr=0.000001, beta_1=0.5, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        # Build and compile the critic
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])
        
        # Build the generator
        self.generator = self.build_generator()
        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        # For the combined model we will only train the generator
        self.critic.trainable=False

        # The critic takes generated images as input and determines validity
        valid = self.critic(img)

        # The combined model (stacked generator and critic)
        self.combined = Model(z, valid)
        #self.parallel_model = multi_gpu_model(self.combined, gpus=self.Ngpus, cpu_relocation=False)
        #self.parallel_model.compile(loss=self.wasserstein_loss,
        #   optimizer=optimizer,
        #   metrics=['accuracy'])
        self.combined.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])
      

    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)
    
    
    def build_generator(self):

        initialize=RandomNormal(mean=0.0, stddev=0.02, seed=None)
        model = Sequential()
        model.add(Dense(1024 * 4 * 4, name='dense_1'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_8'))
        model.add(Activation('relu', name='activation'))
        
        model.add(Reshape((4, 4, 1024), name='reshape_2'))
        
        model.add(Conv2DTranspose(512, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_9'))
        model.add(Activation('relu', name='activation_1'))
        
        model.add(Conv2DTranspose(256, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose_1'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_10'))
        model.add(Activation('relu', name='activation_2'))
        
        model.add(Conv2DTranspose(128, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose_2'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_11'))
        model.add(Activation('relu', name='activation_3'))
        
        model.add(Conv2DTranspose(64, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose_3'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_12'))
        model.add(Activation('relu', name='activation_4'))
        
        model.add(Conv2DTranspose(32, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose_4'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_13'))
        model.add(Activation('relu', name='activation_5'))
        
        model.add(Conv2DTranspose(16, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose_5'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_14'))
        model.add(Activation('relu', name='activation_6'))
        
        model.add(Conv2DTranspose(3, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                                 name='conv2d_transpose_6'))
        model.add(BatchNormalization(momentum=0.8, name ='batch_normalization_15'))
        model.add(Activation('tanh', name='activation_7'))

        #j = 0
        #for i in range(25,28):
        #    w = CAE.layers[i].get_weights()
        #    model.layers[j].set_weights(w)
        #   j =j+1
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        model.load_weights('/home/cc/Data/CAE/CAE_v2_weights.h5', by_name=True)
        Gener = Model(noise, img)
        #Gener = multi_gpu_model(Gener, gpus=self.Ngpus, cpu_relocation=False)
        return Gener
    

    def build_critic(self):
        initialize=RandomNormal(mean=0.0, stddev=0.02, seed=None)
        model = Sequential()
        model.add(Conv2D(16, kernel_size=4, strides=2, kernel_initializer=initialize, input_shape=(512,512,3),
                         padding="same", name ='conv2d'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu'))

        model.add(Conv2D(32, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                         name ='conv2d_1'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization_1'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu_1'))

        model.add(Conv2D(64, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                        name ='conv2d_2'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization_2'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu_2'))

        model.add(Conv2D(128, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                        name ='conv2d_3'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization_3'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu_3'))

        model.add(Conv2D(256, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                        name ='conv2d_4'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization_4'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu_4'))

        model.add(Conv2D(512, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                        name ='conv2d_5'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization_5'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu_5'))

        model.add(Conv2D(1024, kernel_size=4, strides=2, kernel_initializer=initialize, padding="same",
                        name ='conv2d_6'))
        model.add(BatchNormalization(momentum=0.8, name='batch_normalization_6'))
        model.add(LeakyReLU(alpha=0.2, name='leaky_re_lu_6'))

        model.add(Flatten(name='flatten_2'))
        model.add(Dense(1, activation='tanh')) 
        #j=0
        #for i in range(0,22):
        #    w = CAE.layers[i].get_weights()
        #   model.layers[j].set_weights(w)
        #   j = j+1
    
        img = Input(shape=self.img_shape)
        validity = model(img)
        model.load_weights('/home/cc/Data/CAE/CAE_v2_weights.h5', by_name=True,skip_mismatch=True)
        Crit = Model(img, validity)
        #Crit = multi_gpu_model(Crit, gpus=self.Ngpus, cpu_relocation=False)
        return Crit
    
    
    def train(self, epochs, batch_size=128, sample_interval=1):
        # Adversarial ground truths
        valid = -np.ones((self.batch_size, 1)) #Valid is -1
        fake = np.ones((self.batch_size, 1))
        for epoch in range(epochs):
            for _ in range(self.n_critic):
                #print(epoch,_)
                # ---------------------
                #  Train Discriminator
                # ---------------------
                imgs=self.image_generator.next() # Select a random batch of images
                # Sample noise as generator input
                #we do this to prevent error on last batch of epoch
                bb = imgs.shape[0]
                noise = np.random.normal(0, 1, (bb, self.latent_dim))
                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)
                # Train the critic
                valid1 = valid [0:bb]
                fake1 =  fake [0:bb]
                d_loss_real = self.critic.train_on_batch(imgs, valid1)
                d_loss_fake = self.critic.train_on_batch(gen_imgs, fake1)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)   
            # ---------------------
            #  Train Generator
            # ---------------------
            g_loss = self.combined.train_on_batch(noise, valid1)

            # Plot the progress
            if epoch % 100 == 0:
                print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 3, 3
        gen_imgs = self.generator.predict(self.noise)

        # Rescale images from -1 -- 1 to   0 -- 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c , figsize=(15,15))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig('/home/cc/Data/'+testname+'/Samples_GAN_%d.png' % epoch)
        plt.close()

In [ ]:
wgan = WGAN()
steps=(wgan.len)
print(steps)
wgan.train(epochs=100*steps, batch_size=256, sample_interval=200)

Found 236345 validated image filenames.
924
0 [D loss: 0.999997] [G loss: 0.996673]
